In [374]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json
import numpy as np
np.random.seed(102)


In [373]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)

# Using the function to open and load all keys in that file 
api_keys = get_keys("/Users/ASM/Documents/secrets/tmdb_api.json")

# Setting the first (and only) value as a variable
key = list(api_keys.values())[0]

In [642]:
# Getting the list of the movies by revenue(order = descending), 
# using "Discover" TMDB API
# each pages has 20 movies
# year = primary realease year + till today
def discover_request(pages = 1, year=2013):
    pages_list = range(1,pages+1)
    movies_list = []
    movies_list_date = []
    for page in pages_list:
        discover_response = requests.get(f'https://api.themoviedb.org/3/discover/movie?api_key={key}&language=en-US&region=US&sort_by=popularity.desc&include_adult=false&include_video=false&page={page}&release_date.gte={year}&with_original_language=en')
        discover_list = discover_response.json()['results']
        discover_list_id = [i['id'] for i in discover_list]
        discover_list_date = [i['release_date'] for i in discover_list]
        movies_list += discover_list_id
        movies_list_date += discover_list_date
    
    return movies_list, movies_list_date


# using titles from "discover() function"
# getting detailed data for each movie

def movies_request(movies_list):
    movies_list_complete = []
    for title in movies_list:
        response = requests.get(f'https://api.themoviedb.org/3/movie/{title}?api_key={key}&language=en-US')
        movies_list_complete.append(response.json())
    return movies_list_complete



In [546]:
# Uncomment below only if need to run API again:

# movies_list, movies_list_date= discover_request(pages=500,year=2000)

In [547]:
df_id_date = pd.DataFrame(list(zip(movies_list, movies_list_date)), 
               columns =['ID', 'release_date'])

df_id_date['release_date'] = pd.to_datetime(df_id_date.release_date) 
df_id_date['release_date_2'] = df_id_date.release_date.dt.year
df_id_date = df_id_date.sort_values('release_date_2', ascending=False)

In [693]:
list_2000 = list(df_id_date.loc[(df_id_date['release_date_2'] >= 2000)]["ID"])

In [695]:
# Uncomment below only if need to run API again:

# movies_list_complete= movies_request(list_2000)
# movies_list_complete

In [629]:
df= pd.DataFrame(movies_list_complete)

In [630]:
df.to_csv('df_movies.csv')